In [1]:
import pysodb
import pandas as pd
from scipy import sparse

In [2]:
sodb = pysodb.SODB()
adata = sodb.load_experiment('Booeshaghi2021Isoform', 'mouse2_slice229')

load experiment[mouse2_slice229] in dataset[Booeshaghi2021Isoform]


In [8]:
adata

AnnData object with n_obs × n_vars = 5083 × 254
    obs: 'fovID', 'fov_x', 'fov_y', 'volume', 'center_x', 'center_y', 'slice_id', 'sample_id', 'label', 'subclass', 'class_label', 'cell_id', 'leiden'
    var: 'n_iso'
    uns: 'leiden', 'leiden_colors', 'moranI', 'neighbors', 'pca', 'spatial_neighbors', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances', 'spatial_connectivities', 'spatial_distances'

In [3]:
# 1. 保存空间坐标（obsm['spatial']）
# 提取 spatial 数组并构建 DataFrame
spatial_arr = adata.obsm['spatial']

# 计算最小值
x_min = spatial_arr[:, 0].min()
y_min = spatial_arr[:, 1].min()

# 平移坐标
spatial_shifted = spatial_arr - [x_min, y_min]

df_spatial = pd.DataFrame({
    'CELL_ID': adata.obs['cell_id'].values,
    'X': spatial_shifted[:, 0],
    'Y': spatial_shifted[:, 1]
})
# 写出 CSV
df_spatial.to_csv('data/cortex/adata_spatial.csv', index=False)
print("Saved spatial → adata_spatial.csv")

# 2. 保存细胞子类（obs['subclass']）
df_subclass = adata.obs[['cell_id', 'subclass']].copy()
df_subclass.columns = ['CELL_ID', 'SUBCLASS']
df_subclass.to_csv('data/cortex/adata_subclass.csv', index=False)
print("Saved subclass → adata_subclass.csv")

# 3. 保存基因表达矩阵（adata.X）
# 先处理稀疏矩阵
X = adata.X
if sparse.issparse(X):
    X = X.toarray()
# 构建 DataFrame：列名为 var_names，行索引加上 CELL_ID
df_expr = pd.DataFrame(X, columns=adata.var_names)
df_expr.insert(0, 'CELL_ID', adata.obs['cell_id'].values)
# 写出 CSV
df_expr.to_csv('data/cortex/adata_expression.csv', index=False)
print("Saved expression matrix → adata_expression.csv")

Saved spatial → adata_spatial.csv
Saved subclass → adata_subclass.csv
Saved expression matrix → adata_expression.csv
